In [27]:
import polars as pl
import fastexcel as fex
from typing import List
from database import get_db_session
from models import Supplier, Customer
from models.settings import Setting

with open("data/bst jan2025 sales&pur ffb.xlsx", "rb") as data:
    file = data
    sheets = fex.read_excel(data.read()).sheet_names

df = pl.read_excel(file, sheet_name='jan2025')


session = next(get_db_session())
customers = session.query(Customer).all()
customers = [row.__dict__ for row in customers]
suppliers = session.query(Supplier).all()
suppliers = [row.__dict__ for row in suppliers]
setting: Setting = session.query(Setting).first()

df_customer = pl.DataFrame(customers).drop(['_sa_instance_state'])
df_supplier = pl.DataFrame(suppliers).drop(['_sa_instance_state'])

In [73]:
import polars as pl
import xlsxwriter

def fraction_df(df: pl.DataFrame, fraction: int = 100) -> List[pl.DataFrame]:
    if df.shape[0] <= fraction:
        return [df]
    last_code = df[min(fraction, df.shape[0]) - 1, "Code"]
    fraction_indexs = df.select(pl.arg_where(df["Code"] == last_code)).to_series().to_list()
    max_index = max(fraction_indexs) + 1
    min_index = min(fraction_indexs)
    # if max index > fraction, then split the dataframe on min_index - 1
    # else split the dataframe on max_index
    if max_index > fraction:
        return [df[:min_index]] + fraction_df(df[min_index:], fraction)
    return [df[:max_index]] + fraction_df(df[max_index:], fraction)

def fraction_df2(df: pl.DataFrame, fraction: int = 100) -> List[pl.DataFrame]:
    if df.shape[0] <= fraction:
        return [df]

    # Get the last `Code` in the first `fraction` rows
    last_code = df[fraction - 1, "Code"]
    
    # Find all indices where `Code == last_code`
    fraction_indices = df.select(pl.arg_where(df["Code"] == last_code)).to_series().to_list()
    
    # Find the range of that group
    max_index = max(fraction_indices) + 1
    min_index = min(fraction_indices)

    # If the whole `Code` group is larger than `fraction`, we take the first `fraction` rows only
    if max_index - min_index >= fraction:
        max_index = fraction

    # Ensure the first chunk is at least `fraction` rows by expanding into the next `Code`
    while max_index < df.shape[0] and max_index < fraction:
        next_code = df[max_index, "Code"]
        next_code_indices = df.select(pl.arg_where(df["Code"] == next_code)).to_series().to_list()
        max_index = max(next_code_indices) + 1

    return [df[:max_index]] + fraction_df(df[max_index:], fraction)

def purchase_invoice(df: pl.DataFrame, setting: Setting):# -> List[pl.DataFrame]:
    # filter out rows with null values in Date Out and Supplier
    # remove whitespace from Supplier, start, end and in between
    pur = df.filter(
        (df["Date Out"].is_not_null()) & (df["Supplier"].is_not_null())
    ).with_columns(
        Supplier = pl.col("Supplier").str.strip_chars().str.replace_all(r"\s+", "")
    )

    pur = pur.join(df_supplier, left_on="Supplier", right_on="code", how="left")
    # only need columns [Date Out => DocDate, Supplier => Code, Net Wt(Ton) => Qty, Price(ton) => U/Price]
    pur = pur.select([
        "Date Out",
        "Supplier",
        "Net Wt(Ton)",
        "Price(ton)",
        "Gross Amt",
    ]).rename({
        "Date Out": "DocDate",
        "Supplier": "Code",
        "Net Wt(Ton)": "Qty",
        "Price(ton)": "UnitPrice",
        "Gross Amt": "Amount",
    })
    

    start_index = setting.purchase_index
    pur_grouped = (
        pur
        .group_by(
            ["Code"],
            maintain_order=True
        )
        .all()
        .with_row_index(offset=start_index)
        .with_columns(
            Seq = pl.col("DocDate").list.len().map_elements(lambda x: list(range(1, x + 1)), return_dtype=List[int]),
            DocNo = pl.col("index").map_elements(lambda x: f"PI-{x:0>5}"),
        )
        .drop(["index"])
        .explode("DocDate", "Seq", "Qty", "UnitPrice", "Amount")
        .with_columns(
            ItemCode = pl.lit("610-001"),
            Account = pl.lit("610-000"),
        )
    )
    return pur_grouped
g = purchase_invoice(df, setting)
f = fraction_df2(g, 100)
# write f into separate excel sheet
with xlsxwriter.Workbook("output.xlsx") as workbook:
    i = 0
    for dataframes in f:
        dataframes.write_excel(workbook=workbook, worksheet=f"purchaseto_{i}")
        i += 1




sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [3]:
def sale_invoice(df: pl.DataFrame, setting: Setting):
    sal = df.filter(
        (df["Date Out"].is_not_null()) & (df["Supplier"].is_not_null())
    )
    # saperate into tpt and worker then append together
    # column needed for tpt
    # [Date Out => DocDate, Customer => Code, Net Wt(Ton) => Qty, TPT CHRG => UnitPrice, Tpt Amt => Amount]
    # column needed for worker
    # [Date Out => DocDate, Customer => Code, Net Wt(Ton) => Qty, Worker Chrg => UnitPrice, Worker Amt => Amount]
    sal = sal.join(df_customer, left_on="Supplier", right_on="code", how="left")

    tpt = sal.select([
        "Date Out",
        "Supplier",
        "Net Wt(Ton)",
        "TPT CHRG",
        "Tpt Amt",
    ]).rename({
        "Date Out": "DocDate",
        "Supplier": "Code",
        "Net Wt(Ton)": "Qty",
        "TPT CHRG": "UnitPrice",
        "Tpt Amt": "Amount",
    }).drop_nulls(["UnitPrice", "Amount"]).with_columns(
        ItemCode = pl.lit("500-002"),
        Account = pl.lit("500-000"),
    )

    worker = sal.select([
        "Date Out",
        "Supplier",
        "Net Wt(Ton)",
        "Worker Chrg",
        "Worker Amt",
    ]).rename({
        "Date Out": "DocDate",
        "Supplier": "Code",
        "Net Wt(Ton)": "Qty",
        "Worker Chrg": "UnitPrice",
        "Worker Amt": "Amount",
    }).drop_nulls(["UnitPrice", "Amount"]).with_columns(
        ItemCode = pl.lit("500-003"),
        Account = pl.lit("500-000"),
    )
    start_index = setting.sales_index
    combined =(tpt.vstack(worker)
        .group_by(
            ["Code"],
            maintain_order=True
        )
        .all()
        .with_row_index(offset=start_index)
        .with_columns(
            Seq = pl.col("DocDate").list.len().map_elements(lambda x: list(range(1, x + 1)), return_dtype=List[int]),
            # DocNo = f"PI-{pl.col('index'):0>5}"
            DocNo = pl.col("index").map_elements(lambda x: f"IV-{x:0>5}"),
            # ItemCode = "610-001",
            # Account = "610-000"
        )
        .drop(["index"])
        .explode("DocDate", "Seq", "Qty", "UnitPrice", "Amount", "ItemCode", "Account")
    )

    return combined

sale_invoice(df, setting)
# df.columns

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


Code,DocDate,Qty,UnitPrice,Amount,ItemCode,Account,Seq,DocNo
str,date,f64,i64,f64,str,str,i64,str
"""1018""",2025-01-12,1.31,40,52.4,"""500-002""","""500-000""",1,"""IV-00002"""
"""1026""",2025-01-23,0.36,20,20.0,"""500-002""","""500-000""",1,"""IV-00003"""
"""1034""",2025-01-06,0.3,50,50.0,"""500-002""","""500-000""",1,"""IV-00004"""
"""1039""",2025-01-24,0.4,28,28.0,"""500-002""","""500-000""",1,"""IV-00005"""
"""2019""",2025-01-04,0.81,18,14.58,"""500-002""","""500-000""",1,"""IV-00006"""
…,…,…,…,…,…,…,…,…
"""2114""",2025-01-11,2.01,90,180.9,"""500-003""","""500-000""",4,"""IV-00071"""
"""2119""",2025-01-02,0.97,15,14.55,"""500-002""","""500-000""",1,"""IV-00072"""
"""2119""",2025-01-02,0.97,70,67.9,"""500-003""","""500-000""",2,"""IV-00072"""
